<a href="https://colab.research.google.com/github/heber-augusto/desafio-7-2020/blob/master/data_collect_using_mqtt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/repos/desafio-7-2020


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/repos/desafio-7-2020


In [2]:
pip install paho-mqtt

     |████████████████████████████████| 102kB 2.5MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.0-cp36-none-any.whl size=61415 sha256=79a6d564339a6a690c1c85cbbd5e67614071bf323b6403624de214e1c563ae99
  Stored in directory: /root/.cache/pip/wheels/02/94/6c/8474137cb7a5a3e001d70a22c8ff919caee69435376bccce79
Successfully built paho-mqtt


In [ ]:
import paho.mqtt.client as mqtt
import json
import csv

all_rows_ids = []
csv_file = open('some.csv', 'w', newline='')
writer = None

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("tnt")



# The callback for when the client receives a CONNACK response from the server.
def on_disconnect(client, userdata, rc):
    if rc != 0:
        print(f"Unexpected disconnection: {rc}")
       


# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    global all_rows_ids
    global csv_file
    global writer

    current_row    = json.loads(msg.payload)
    current_row_id = current_row['row']
    # 17016 is the total number of rows. call diconnect() when finish 
    # saving all rows
    if len(all_rows_ids) >= 17016:
        client.disconnect()
        return

    # at the first row, save the header
    if len(all_rows_ids) == 0:
        field_names = list(current_row.keys())
        writer = csv.DictWriter(csv_file, fieldnames=field_names)
        writer.writeheader()        

    # if the id is already saved, return without including it at the file
    if current_row_id in all_rows_ids:
        return

    # append the current row to a list that is used to check repetition
    all_rows_ids.append(current_row_id)

    # create the row into the file
    writer.writerow(current_row)

    # print status for each 100 rows
    if (len(all_rows_ids) % 100) == 0:
        print(f'coletados {len(all_rows_ids)} registros')

    # print(f'{msg.topic} {current_row_id}: {current_row}')

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.on_disconnect = on_disconnect
client.username_pw_set('maratoners','ndsjknvkdnvjsbvj')
client.reconnect_delay_set(1,120)
print(client.connect("tnt-iot.maratona.dev", 30573, 60))

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()
csv_file.close()
print(f'coletados {len(all_rows_ids)} registros')

0
Connected with result code 0
